### Word Group In Youtube

In [83]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [84]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [85]:
# language pair
lang_folder = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 1000  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:1000
file_ext:6
unique_word_ratio:80
include_wordgroup_ratio:80
word_usage_min:10
word_usage_max:100


In [86]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [87]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [88]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [89]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [90]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [91]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [92]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
552149,fruitcocktail,6
552150,andthesunlightshining,6
552151,upravo,6
552152,yagawa,6


In [93]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [94]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
995,tea,179415
996,quit,178993
997,bloody,178894
998,planet,178653


In [95]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [96]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [97]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

997

#### Youtube Sentence Analysis

In [98]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:29.416,00:00:33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,00:00:33.810,00:00:36.876,shivering cold and snow,tPffr9WPl9Y
2,00:00:37.150,00:00:39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,00:00:39.257,00:00:43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,00:00:43.796,00:00:50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...
5742563,00:02:59.400,00:03:03.030,wounded and,78VHQeebbWM
5742564,00:03:03.030,00:03:10.590,general staff of the armed forces of ukraine o...,78VHQeebbWM
5742565,00:03:10.590,00:03:13.159,facebook on december,78VHQeebbWM
5742566,00:03:13.159,00:03:16.769,you can express your thoughts in the comments ...,78VHQeebbWM


In [99]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [100]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM


In [101]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [102]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [103]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,37.5,"jingle, bell","way, all, the"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,50.0,"shivering, snow","and, cold"
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,80.0,"season, festive","you, wish, do, what, for, else, this, in"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,50.0,"lights, edinburgh, colours, decorated, herself","has, the, with, and, christmas, of, city"
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,70.6,"edinburgh, places, experience, explore, celebr...","here, so, we, today, how, will, being, and, di..."
...,...,...,...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM,50.0,wounded,and
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,50.0,"operational, forces, staff, shared, armed, ukr...","been, has, the, information, on, of, general"
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM,33.3,"december, facebook",on
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,55.5,"express, section, comments, thoughts","can, the, your, you, in"


In [104]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,37.5,"jingle, bell","way, all, the"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,50.0,"shivering, snow","and, cold"
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,80.0,"season, festive","you, wish, do, what, for, else, this, in"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,50.0,"lights, edinburgh, colours, decorated, herself","has, the, with, and, christmas, of, city"
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,70.6,"edinburgh, places, experience, explore, celebr...","here, so, we, today, how, will, being, and, di..."
...,...,...,...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM,50.0,wounded,and
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,50.0,"operational, forces, staff, shared, armed, ukr...","been, has, the, information, on, of, general"
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM,33.3,"december, facebook",on
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,55.5,"express, section, comments, thoughts","can, the, your, you, in"


In [105]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,37.5,"jingle, bell","way, all, the"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,50.0,"shivering, snow","and, cold"
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,80.0,"season, festive","you, wish, do, what, for, else, this, in"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,50.0,"lights, edinburgh, colours, decorated, herself","has, the, with, and, christmas, of, city"
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,70.6,"edinburgh, places, experience, explore, celebr...","here, so, we, today, how, will, being, and, di..."
...,...,...,...,...,...,...,...,...
5589565,5742563,179.400,183.030,wounded and,78VHQeebbWM,50.0,wounded,and
5589566,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,50.0,"operational, forces, staff, shared, armed, ukr...","been, has, the, information, on, of, general"
5589567,5742565,190.590,193.159,facebook on december,78VHQeebbWM,33.3,"december, facebook",on
5589568,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,55.5,"express, section, comments, thoughts","can, the, your, you, in"


In [106]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-106-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,80.0,"season, festive","you, wish, do, what, for, else, this, in",what else do you wish for in this
7,7,62.231,66.363,people of edinburgh wait for these few days th...,tPffr9WPl9Y,81.8,"throughout, edinburgh","these, the, wait, year, few, days, of, for, pe...",wait for these few days
8,8,66.866,70.225,so there is no point of sitting in home in thi...,tPffr9WPl9Y,85.7,NaN,"there, so, point, year, of, is, home, no, time...",so there is no point of sitting in home in thi...
10,10,77.251,81.167,beautiful lights make the city like a fantasy ...,tPffr9WPl9Y,77.8,"lights, fantasy","the, make, land, like, a, beautiful, city",make the city like a
12,12,99.101,104.558,apart from christmas lights different projecte...,tPffr9WPl9Y,56.2,"lights, scenaries, projected, apart, attractive","bring, to, life, being, different, christmas, ...",are being used to bring life to
...,...,...,...,...,...,...,...,...,...
5589535,5742532,28.410,34.590,naivev says that the situation in the northern...,78VHQeebbWM,61.5,"zone, northern, operational, naivev","situation, says, the, control, that, under, is...",says that the situation in the
5589539,5742536,48.860,54.010,the work we are doing should increase the resi...,78VHQeebbWM,60.0,"resilience, rationality, defences, increase","work, the, our, and, should, doing, are, of, we",the work we are doing should
5589544,5742541,72.090,77.909,for the evacuation of citizens who are current...,78VHQeebbWM,55.6,"dnipro, currently, river, evacuation, citizens","bank, who, the, on, of, are, for, a, left, in",on the left bank of the
5589552,5742549,109.030,113.009,press office of the security service of ukrain...,78VHQeebbWM,55.5,"reported, ukraine, press","security, service, the, office, of",office of the security service of


In [107]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,what else do you wish for in this,37.150000,38.460417,what else do you wish for in this festive season,tPffr9WPl9Y
1,wait for these few days,63.477159,65.116841,people of edinburgh wait for these few days th...,tPffr9WPl9Y
2,so there is no point of sitting in home in thi...,66.866000,70.225000,so there is no point of sitting in home in thi...,tPffr9WPl9Y
3,make the city like a,78.504120,80.227160,beautiful lights make the city like a fantasy ...,tPffr9WPl9Y
4,are being used to bring life to,101.855000,103.538000,apart from christmas lights different projecte...,tPffr9WPl9Y
...,...,...,...,...,...
2323876,says that the situation in the,28.873500,31.345500,naivev says that the situation in the northern...,78VHQeebbWM
2323877,the work we are doing should,48.860000,50.557159,the work we are doing should increase the resi...,78VHQeebbWM
2323878,on the left bank of the,75.193467,76.809856,for the evacuation of citizens who are current...,78VHQeebbWM
2323879,office of the security service of,109.385268,111.872143,press office of the security service of ukrain...,78VHQeebbWM


In [108]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,what else do you wish for in this,36.850000,38.760417,what else do you wish for in this festive season,tPffr9WPl9Y
1,wait for these few days,63.177159,65.416841,people of edinburgh wait for these few days th...,tPffr9WPl9Y
2,so there is no point of sitting in home in thi...,66.566000,70.525000,so there is no point of sitting in home in thi...,tPffr9WPl9Y
3,make the city like a,78.204120,80.527160,beautiful lights make the city like a fantasy ...,tPffr9WPl9Y
4,are being used to bring life to,101.555000,103.838000,apart from christmas lights different projecte...,tPffr9WPl9Y
...,...,...,...,...,...
2323876,says that the situation in the,28.573500,31.645500,naivev says that the situation in the northern...,78VHQeebbWM
2323877,the work we are doing should,48.560000,50.857159,the work we are doing should increase the resi...,78VHQeebbWM
2323878,on the left bank of the,74.893467,77.109856,for the evacuation of citizens who are current...,78VHQeebbWM
2323879,office of the security service of,109.085268,112.172143,press office of the security service of ukrain...,78VHQeebbWM


In [109]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,what else do you wish for in this,37,39,what else do you wish for in this festive season,tPffr9WPl9Y
1,wait for these few days,63,65,people of edinburgh wait for these few days th...,tPffr9WPl9Y
2,so there is no point of sitting in home in thi...,67,71,so there is no point of sitting in home in thi...,tPffr9WPl9Y
3,make the city like a,78,81,beautiful lights make the city like a fantasy ...,tPffr9WPl9Y
4,are being used to bring life to,102,104,apart from christmas lights different projecte...,tPffr9WPl9Y
...,...,...,...,...,...
2323876,says that the situation in the,29,32,naivev says that the situation in the northern...,78VHQeebbWM
2323877,the work we are doing should,49,51,the work we are doing should increase the resi...,78VHQeebbWM
2323878,on the left bank of the,75,77,for the evacuation of citizens who are current...,78VHQeebbWM
2323879,office of the security service of,109,112,press office of the security service of ukrain...,78VHQeebbWM


In [110]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,i would say it was kind of long like i remembe...,1317,1330,elizabeth finley um but yeah i had a really go...,iJkfTPN8MS0
1,so i mean i actually think this is the group t...,3336,3349,theresa anderson much harder so i mean i actua...,iZcte874ICY
2,you know people say stuff out of their face th...,1239,1248,leroy ricksy jr and especially now with the el...,9nBekn1eQtg
3,and then the other one was sick and so i was l...,2932,2941,michaela martin and even my backup childcare b...,twJkEtTvvEo
4,i think this is the key i think its really imp...,1609,1626,i think this is the key i think its really imp...,4bPfMqlzfWE
...,...,...,...,...,...
2323876,so y l a a,817,819,is a set of measured covariates well use a pot...,vM_gw1JpSKU
2323877,d y by d t,1542,1544,d y by d t plus m times d squared y by d t squ...,Go6ak2ms1U4
2323878,l d l d is,1301,1303,to impose l d l d is liquidated damages that i...,JAb5_hmdVHw
2323879,o a a a a,89,90,o a a a a u u g,3KsbG3WCQuk


In [111]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,i would say it was kind of long like i remembe...,1317,1330,elizabeth finley um but yeah i had a really go...,iJkfTPN8MS0,70
1,so i mean i actually think this is the group t...,3336,3349,theresa anderson much harder so i mean i actua...,iZcte874ICY,73
2,you know people say stuff out of their face th...,1239,1248,leroy ricksy jr and especially now with the el...,9nBekn1eQtg,82
3,and then the other one was sick and so i was l...,2932,2941,michaela martin and even my backup childcare b...,twJkEtTvvEo,74
4,i think this is the key i think its really imp...,1609,1626,i think this is the key i think its really imp...,4bPfMqlzfWE,69
...,...,...,...,...,...,...
2323876,so y l a a,817,819,is a set of measured covariates well use a pot...,vM_gw1JpSKU,80
2323877,d y by d t,1542,1544,d y by d t plus m times d squared y by d t squ...,Go6ak2ms1U4,80
2323878,l d l d is,1301,1303,to impose l d l d is liquidated damages that i...,JAb5_hmdVHw,60
2323879,o a a a a,89,90,o a a a a u u g,3KsbG3WCQuk,40


In [112]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-112-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-112-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,you know people say stuff out of their face th...,1239,1248,leroy ricksy jr and especially now with the el...,9nBekn1eQtg
1,just looking at this list you probably might b...,1107,1116,marcy ramirez so i think that if you feel that...,d5XwbH3Tr68
2,with dr king many people thought that that was...,979,988,craig robinson standing out in that mall with ...,G3w3f6M_w54
3,say well if we can try this then maybe well ta...,2945,2955,joseph john curto np sometimes ill say well if...,U33ldI3uSmk
4,it was just like a very interesting day and i ...,2757,2766,elizabeth finley and yeah i im not going to de...,iJkfTPN8MS0
...,...,...,...,...,...
2157276,if i go to a,1057,1059,if i go to a blog post and turn on the use fea...,yTreRQiccVg
2157277,of a t or an,965,967,for example if im looking at an air fitting if...,f39NXqeCZ_w
2157278,if i go to a,2823,2825,if i go to a peering here you will see cf,G8NFGMu641c
2157279,so if i do a,3532,3534,so if i do a microscopy experiment,cKEZG76bNsQ


In [113]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-113-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,you know people say stuff out of their face th...,1239,1248,leroy ricksy jr and especially now with the el...,9nBekn1eQtg,73
1,just looking at this list you probably might b...,1107,1116,marcy ramirez so i think that if you feel that...,d5XwbH3Tr68,69
2,with dr king many people thought that that was...,979,988,craig robinson standing out in that mall with ...,G3w3f6M_w54,80
3,say well if we can try this then maybe well ta...,2945,2955,joseph john curto np sometimes ill say well if...,U33ldI3uSmk,67
4,it was just like a very interesting day and i ...,2757,2766,elizabeth finley and yeah i im not going to de...,iJkfTPN8MS0,49
...,...,...,...,...,...,...
2157276,if i go to a,1057,1059,if i go to a blog post and turn on the use fea...,yTreRQiccVg,14
2157277,of a t or an,965,967,for example if im looking at an air fitting if...,f39NXqeCZ_w,13
2157278,if i go to a,2823,2825,if i go to a peering here you will see cf,G8NFGMu641c,29
2157279,so if i do a,3532,3534,so if i do a microscopy experiment,cKEZG76bNsQ,35


In [114]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-114-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,now this shot could have been the top shot of ...,199,206,now this shot could have been the top shot of ...,WTBUdOIqOVs
1,so you can imagine if you go through all of th...,293,306,so you can imagine if you go through all of th...,K1GhQ8633Bg
2,and so one of the questions people say to me i...,971,978,craig robinson and so one of the questions peo...,G3w3f6M_w54
3,its a little family that you turn into and jus...,49,59,its a little family that you turn into and jus...,GymR27z8RHM
4,and i remember when it happened i was like oh ...,796,803,ayanna howard and i remember when it happened ...,M_IRnJZ2xQ0
...,...,...,...,...,...
471704,so if i go to,2831,2833,so if i go to,W0Yj7W3HjYQ
471705,if we had a d,2359,2361,if we had a d,q7clXE12JVA
471706,in a way i am,4366,4368,in a way i am,5f5cjvTRokY
471707,i was on a t,3265,3270,i was on a t,AEUKkluZUls


In [115]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,started right here one year ago today at,zyrp9DRjAII
1,which just goes through how these things work,zyHhANpL2xk
2,up going completely wrong but at least they tr...,zyEMeStyuB8
3,pretty good clear time yesterday actually um l...,zyEMeStyuB8
4,do ya ever wan na get married,zxcNJO9nlsw
...,...,...
2013,times thanks very much take care,09JnjRZdldU
2014,these feel absolutely perfect so were just gon...,08hVBW8kCrE
2015,okay let me make another simple point,06axJ1girNM
2016,she set up her first company at age,03pI5Jlz3C0


In [116]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,now this shot could have been the top shot of ...,199,206,now this shot could have been the top shot of ...,WTBUdOIqOVs
1,so you can imagine if you go through all of th...,293,306,so you can imagine if you go through all of th...,K1GhQ8633Bg
2,and so one of the questions people say to me i...,971,978,craig robinson and so one of the questions peo...,G3w3f6M_w54
3,its a little family that you turn into and jus...,49,59,its a little family that you turn into and jus...,GymR27z8RHM
4,and i remember when it happened i was like oh ...,796,803,ayanna howard and i remember when it happened ...,M_IRnJZ2xQ0
...,...,...,...,...,...
1971,got ta pull em out,151,152,got ta pull em out,OY9g0rATphc
1972,uh jack got ta go,60,62,uh jack got ta go,GkTtP_D1G5g
1973,good d great re t,801,813,good d great re t,fegm_wVWjnY
1974,go say out mm hmm,515,517,go say out mm hmm,mJcQVhwf4aQ


In [117]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,now this shot could have been the top shot of ...,199,206,now this shot could have been the top shot of ...,WTBUdOIqOVs,https://www.youtube.com/watch?v=WTBUdOIqOVs&t=...
1,so you can imagine if you go through all of th...,293,306,so you can imagine if you go through all of th...,K1GhQ8633Bg,https://www.youtube.com/watch?v=K1GhQ8633Bg&t=...
2,and so one of the questions people say to me i...,971,978,craig robinson and so one of the questions peo...,G3w3f6M_w54,https://www.youtube.com/watch?v=G3w3f6M_w54&t=...
3,its a little family that you turn into and jus...,49,59,its a little family that you turn into and jus...,GymR27z8RHM,https://www.youtube.com/watch?v=GymR27z8RHM&t=49s
4,and i remember when it happened i was like oh ...,796,803,ayanna howard and i remember when it happened ...,M_IRnJZ2xQ0,https://www.youtube.com/watch?v=M_IRnJZ2xQ0&t=...
...,...,...,...,...,...,...
1971,got ta pull em out,151,152,got ta pull em out,OY9g0rATphc,https://www.youtube.com/watch?v=OY9g0rATphc&t=...
1972,uh jack got ta go,60,62,uh jack got ta go,GkTtP_D1G5g,https://www.youtube.com/watch?v=GkTtP_D1G5g&t=60s
1973,good d great re t,801,813,good d great re t,fegm_wVWjnY,https://www.youtube.com/watch?v=fegm_wVWjnY&t=...
1974,go say out mm hmm,515,517,go say out mm hmm,mJcQVhwf4aQ,https://www.youtube.com/watch?v=mJcQVhwf4aQ&t=...


In [118]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [119]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [120]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [121]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['English_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx']

In [122]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [123]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass